## Imports
We have 3 important files for this purpose and each and every dependency, class, import, function, variable etc is being imported from these modules

1. `detection_helpers` which I made to wrap the original `YOLOv-7` code along with helper functions
2. `tracking_helpers` has modular code which is used to wrap the `DeepSORT` repo and workings
3. `bridge_wrapper` acts as a bridge to bind **ANY** detection model with `DeepSORT`

In [1]:
from detection_helpers import *
from tracking_helpers import *
from  bridge_wrapper2 import *
from PIL import Image

/opt/homebrew/Caskroom/miniconda/base/envs/carenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Detection
Detect objects using `Yolov-7`

In [8]:
detector = Detector(classes = None) # it'll detect ONLY [person,horses,sports ball]. class = None means detect all classes. List info at: "data/coco.yaml"
detector.load_model('./last.pt',) # pass the path to the trained weight file


Fusing layers... 
 Convert model to Traced-model... 
 traced_script_module saved! 
 model is traced! 



# Tracking

Works as follows:
1. Read each frame of video using `OpenCV`
2. Get Bounding Box or Detections from the model per frame
3. Crop those patches and pass on to `reID` model for re identification which is a part of `DeepSORT` method
4. Get the above embeddings and then use `Kalman Filtering` and `Hungerian assignment` to assign the correct BB to the respective object
5. Show, Save the frames

In [9]:
# Initialise  class that binds detector and tracker in one class
tracker = YOLOv7_DeepSORT(reID_model_path="./deep_sort/model_weights/mars-small128.pb", detector=detector)

# output = None will not save the output video
tracker.track_video("./IO_data/input/video/cctv3-2/CCTV_3_2.mp4", output="./IO_data/output/street6.mp4", show_live = False, skip_frames = 0, count_objects = True, verbose=1)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


Processed frame no: 1 || Current FPS: 2.22 || Objects tracked: 7
Processed frame no: 2 || Current FPS: 4.79 || Objects tracked: 6
Processed frame no: 3 || Current FPS: 4.71 || Objects tracked: 6
Processed frame no: 4 || Current FPS: 4.31 || Objects tracked: 7
Processed frame no: 5 || Current FPS: 2.32 || Objects tracked: 7
Processed frame no: 6 || Current FPS: 2.52 || Objects tracked: 7
Processed frame no: 7 || Current FPS: 2.32 || Objects tracked: 7
Processed frame no: 8 || Current FPS: 2.44 || Objects tracked: 7
Processed frame no: 9 || Current FPS: 2.44 || Objects tracked: 7
Processed frame no: 10 || Current FPS: 2.35 || Objects tracked: 7
Processed frame no: 11 || Current FPS: 2.3 || Objects tracked: 7
Processed frame no: 12 || Current FPS: 2.36 || Objects tracked: 7
Processed frame no: 13 || Current FPS: 2.49 || Objects tracked: 7
Processed frame no: 14 || Current FPS: 2.46 || Objects tracked: 7
Processed frame no: 15 || Current FPS: 2.63 || Objects tracked: 7
Processed frame no: 

: 

## Scripts for handling Videos on `Colab / Jupyter Notebook`

### Download a video from Youtube

In [ ]:
# ! pip install pytube
from pytube import YouTube 

link = "https://www.youtube.com/watch?v=kYIf8I1dvdo"
yt = YouTube(link)  

try:
    yt.streams.filter(progressive = True, file_extension = "mp4", resolution = "720p").first().download(output_path = "./", filename = "test.mp4",)
except Exception as e: print(e)

### Trim an existing video

In [ ]:
# ! pip install moviepy
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

ffmpeg_extract_subclip("test.mp4", 10, 100, targetname="trim.mp4") # trim from 10th second to 100th second

### Show an MP4 video Notebook

In [ ]:
from base64 import b64encode

mp4 = open('trim.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""<video width=400 controls><source src="%s" type="video/mp4"></video>""" % data_url)